In [1]:
import pandas as pd
import numpy as np
import copy as cp

## Import and Merge Data

In [2]:
onedrive_path = "C:/Users/cfowle/The Estée Lauder Companies Inc/TeamAnis - General/"

In [3]:
reviews = pd.read_csv(onedrive_path + "Data/Ratings and Reviews/reviews_demand_subcat.csv")
cc      = pd.read_csv(onedrive_path + "Data/Consumer Care/cc_product_month.csv")

C:\Users\cfowle\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
products = pd.read_csv(onedrive_path + "Data/Product/product_codes.csv")

C:\Users\cfowle\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (9,12,23,26,28) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
print(len(reviews))
reviews = reviews.drop(["item_description"], axis = 1).drop_duplicates()
print(len(reviews))

136630
122187


In [6]:
cc = cc.rename(columns = {"Date Month": "date", "P4": "itemid_4", "Brand Clean": "elc_brand"})
products = products.rename(columns = {"P4": "itemid_4", "brand": "elc_brand", "SubCategory":"sub_category"})
products = products[["elc_brand", "itemid_4", 'Major Category', 'Application', 'Category',
       'sub_category']].drop_duplicates(["elc_brand", "itemid_4"])

In [7]:
cc_product = cc.merge(products, how = "left", on = ["itemid_4", "elc_brand"])

In [8]:
cc_reviews_product = reviews.merge(cc_product, how = "left").fillna(0)
cc_reviews_product = cc_reviews_product.groupby(["elc_brand", "date", "sub_category"]).sum().reset_index()

In [9]:
cc_reviews_product = cc_reviews_product.sort_values("date")

## Feature Engineering

### Target Encoding

In [10]:
subcat_avg = cc_reviews_product.groupby("sub_category")["demand"].rolling(6).mean().reset_index()
subcat_avg = subcat_avg.set_index('level_1')
cc_reviews_product["avg_subcat_demand"] = subcat_avg['demand']

In [11]:
brand_avg = cc_reviews_product.groupby("elc_brand")["demand"].rolling(6).mean().reset_index()
brand_avg = brand_avg.set_index('level_1')
cc_reviews_product["avg_brand_demand"] = brand_avg['demand']

In [12]:
cc_reviews_product["month"] = [x[-2:] for x in cc_reviews_product["date"].values]
cc_reviews_product["year"]  = [x[:4] for x in cc_reviews_product["date"].values]

In [13]:
month_map = cc_reviews_product.groupby(["month", "year"])["demand"].mean().reset_index()
month_map = month_map.rename(columns = {"demand": "monthly_avg_demand"})
month_map["avg_12_month_demand"] = month_map.rolling(12).mean()["monthly_avg_demand"]

In [14]:
cc_reviews_product = cc_reviews_product.merge(month_map)

### Relative R&R Data

In [15]:
non_customer_data = cc_reviews_product[["demand_F1", "demand", "demand_P2", "demand_P1",
                                   "avg_subcat_demand", "monthly_avg_demand", "avg_12_month_demand", "avg_brand_demand",
                                   "month", "year"]]

In [16]:
relative_data = cp.deepcopy(non_customer_data)
subcat = cc_reviews_product ##just here so I can copy some old code easily

In [17]:
relative_data["percent_1"] = subcat["rating_1"]/subcat["nb_reviews"]
relative_data["percent_2"] = subcat["rating_2"]/subcat["nb_reviews"]
relative_data["percent_3"] = subcat["rating_3"]/subcat["nb_reviews"]
relative_data["percent_4"] = subcat["rating_4"]/subcat["nb_reviews"]
relative_data["percent_5"] = subcat["rating_5"]/subcat["nb_reviews"]

In [18]:
relative_data["percent_negative"] = subcat['sentiment_negative']/subcat["nb_reviews"]
relative_data["percent_neutral"] = subcat['sentiment_neutral']/subcat["nb_reviews"]
relative_data["percent_positive"] = subcat['sentiment_positive']/subcat["nb_reviews"]

In [19]:
relative_data = relative_data.fillna(0)

In [20]:
relative_data

,demand_F1,demand,demand_P2,demand_P1,avg_subcat_demand,monthly_avg_demand,avg_12_month_demand,avg_brand_demand,month,year,percent_1,percent_2,percent_3,percent_4,percent_5,percent_negative,percent_neutral,percent_positive
0,195.0,0,0.0,0.0,0.000000,25517.980220,33970.711538,0.000000,12,2014,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,1349.0,2342,1889.0,954.0,0.000000,25517.980220,33970.711538,0.000000,12,2014,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,1601.0,1882,2091.0,1999.0,0.000000,25517.980220,33970.711538,0.000000,12,2014,0.000000,0.000000,1.000000,0.000000,0.000000,0.333333,0.000000,0.666667
3,10197.0,4214,5703.0,8153.0,0.000000,25517.980220,33970.711538,0.000000,12,2014,0.000000,0.000000,0.000000,0.000000,1.000000,0.222222,0.333333,0.444444
4,80873.0,112232,95013.0,91494.0,0.000000,25517.980220,33970.711538,0.000000,12,2014,0.090909,0.000000,0.000000,0.060606,0.848485,0.051178,0.115733,0.833088
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29570,3372.0,3933,5084.0,9414.0,712.500000,6170.356044,27236.066300,3815.833333,04,2020,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
29571,1826.0,153,1086.0,3131.0,2240.500000,6170.356044,27236.066300,3841.333333,04,2020,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
29572,0.0,0,0.0,0.0,712.500000,6170.356044,27236.066300,3652.833333,04,2020,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
29573,4562.0,2493,23632.0,31641.0,14607.666667,6170.356044,27236.066300,680.333333,04,2020,0.022727,0.090909,0.022727,0.045455,0.818182,0.021970,0.098864,0.879167


In [21]:
relative_data.to_csv(onedrive_path + "Data/Combined/relative_rr_data_w_target_encodings.csv")

### Relative CC Data

In [22]:
full_relative_data = cp.deepcopy(relative_data)
subcat["cc_tot"] = subcat['Complaints'] + subcat['Compliments'] + subcat['Questions'] + subcat['Suggestions']
cc_counts = cc.drop(['Product Code', 'itemid_4', 'elc_brand', "date"], axis = 1).columns

In [23]:
for col in cc_counts:
    full_relative_data["percent_" + col] = subcat[col]/subcat["cc_tot"]

In [24]:
full_relative_data = full_relative_data.fillna(0)

In [25]:
full_relative_data.to_csv(onedrive_path + "Data/Combined/relative_rr_cc_data_w_target_encodings.csv")